In [13]:
import os
import requests
import json
import openai
import pickle as pkl
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer
import tiktoken

In [147]:
with open('Data/KG_with_ConceptNet_Two_hop.pkl', 'rb') as f:
    IAB_KG=pkl.load(f)

In [3]:
with open('Data/KG_with_ConceptNet_One_hop.pkl', 'rb') as f:
    IAB_KG=pkl.load(f)

In [4]:
with open('Data/MAF_KG.pkl', 'rb') as f1:
    MAF_KG=pkl.load(f1)

In [5]:
IAB_KG[0]

('head_south', 'is_related_to', 'Birds')

In [6]:
MAF_KG[0]

('brassware', 'is_related_to', 'brass')

In [148]:
head=[]
relation=[]
tail=[]
for items in tqdm(IAB_KG):
    head.append(items[0])
    relation.append(items[1])
    tail.append(items[2])
dic={"IAB Keyword": tail, "Relation": relation, "Object": head}
df_IAB_KG=pd.DataFrame.from_dict(dic)

100%|██████████| 835757/835757 [00:00<00:00, 2652270.90it/s]


In [150]:
head=[]
relation=[]
tail=[]
for items in tqdm(MAF_KG):
    head.append(items[0])
    relation.append(items[1])
    tail.append(items[2])
dic={"MAF Keyword": tail, "Relation": relation, "Object": head}
df_MAF_KG=pd.DataFrame.from_dict(dic)

100%|██████████| 181703/181703 [00:00<00:00, 1734616.46it/s]


In [152]:
df_initialKG_only_H=pd.read_csv('data/InitialKG_Only_H.csv')

In [153]:
IAB_keywords=df_initialKG_only_H.columns.to_list()

In [154]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [155]:
df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [156]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [157]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [158]:
maf=df_MAF['item'].to_list()

In [159]:
call=iab_labels+maf

In [160]:
len(call)

1102

In [161]:
print(call[0:10])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel']


In [162]:
tokenizer = tiktoken.get_encoding("cl100k_base")

In [163]:
df_IAB.head

<bound method NDFrame.head of                    IAB_Label  node_ids
0                  Hair Care      1547
1                    Shaving      6342
2                Motorcycles     20159
3        Women's Formal Wear     11540
4           Women's Clothing       729
..                       ...       ...
155                Education       674
156                     Cats      4362
157        Primary Education     23605
158     Books and Literature       460
159  Non-Alcoholic Beverages     62645

[160 rows x 2 columns]>

In [164]:
#create doc query
expanded_call=[]

for items in tqdm(iab_labels):
    list_items=df_IAB_KG[df_IAB_KG['IAB Keyword']==items]['Object'].tolist()
    string_list=" ".join(list_items)
    length=len(tokenizer.encode(string_list))
    if length>=8192:
        print(items)
    expanded_call.append(string_list)



100%|██████████| 160/160 [00:09<00:00, 17.50it/s]


In [165]:
for items in tqdm(maf):
    list_items=df_MAF_KG[df_MAF_KG['MAF Keyword']==items]['Object'].tolist()
    string_list=" ".join(list_items)
    length=len(tokenizer.encode(string_list))
    if length>=8192:
        print(items)
    expanded_call.append(string_list)

  0%|          | 0/942 [00:00<?, ?it/s]

100%|██████████| 942/942 [00:08<00:00, 105.07it/s]


In [241]:
expanded_call[21]

'university action_performed_by_animals daylight greensman gardening green_roof brandscaping xeriscaping streetscaping find_outside intelligent_agent_activity landscape_gardening lawnmower weedeat parkomania lilyturf hard_landscaping rosebush garden artifact greensperson making yardwork landscape weedeating rock_garden potscaping'

In [169]:
total_embeddings=[]

In [170]:
def create_embeddings(item,total_embeddings):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

    

In [171]:
for items in tqdm(expanded_call):
    create_embeddings(items,total_embeddings)

100%|██████████| 1102/1102 [03:03<00:00,  6.01it/s]


In [172]:
len(total_embeddings)

1102

In [173]:
iab_embeddings=total_embeddings[0:160]

In [174]:
maf_embeddings=total_embeddings[160:]

In [175]:
def calculate_precision_recall(true_labels, predictions):
    true_positives = sum(1 for pred in predictions if pred in true_labels)
    false_positives = len(predictions) - true_positives
    false_negatives = len(true_labels) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

In [176]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [177]:
nodes=df_MAF['item'].to_list()
node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
H_Mappings=df_MAF['H_Scores'].to_list()

In [178]:
cos_sim=cosine_similarity(np.array(iab_embeddings[90]).reshape(1,-1),np.array(maf_embeddings[12]).reshape(1,-1))
print(cos_sim)

[[0.74721684]]


In [229]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_node_ids=[]
top_n_IAB_Candidates=[]
top_n_H_scores=[]
top_n_labels=[]
top_n_cosines=[]

for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+j],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)


    #threshold logic needed
    similarities=[x for x in similarities if x[2]>=0.68]

    
    
    if len(similarities)==0:
        similarities.append(['No_Pred','No_Pred',[[0]]])
    
    temp_iab_labels=[]
    temp_cosines=[]


    for items in similarities:
        #print(items[1])
        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])
        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0])
        top_n_nodes.append(nodes[i])
        top_n_node_ids.append(node_id[i])
        top_n_IAB_Candidates.append(IAB_Candidates[i])
        top_n_H_scores.append(H_Mappings[i])


    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)


100%|██████████| 942/942 [01:05<00:00, 14.30it/s]


In [230]:
len(top_n_labels)

942

In [231]:
len(top_n_labels_new)

147512

In [232]:
len(top_n_cosines)

942

In [233]:
MAF_predictions=df_MAF['IAB_Candidates'].to_list()

In [234]:
candidate_count=0
hit_count=0
precision=0
recall=0
count=0
f1=0

for predictions, labels in zip(MAF_predictions,top_n_labels):
    count+=1
    proper=ast.literal_eval(predictions)
    prec, reca=calculate_precision_recall(predictions, labels)

    f=calculate_f1_score(prec, reca)
    precision+=prec
    recall+=reca
    f1+=f


    candidate_count+=len(proper)
    for items in labels:
        if items in proper:
            hit_count+=1

In [235]:
hit_count/candidate_count

0.9148372445117335

In [236]:
precision/942

0.017185849669570204

In [237]:
recall/942

0.07356003319116432

In [238]:
f1/942

0.022420642113156877

In [104]:
dic={'item':top_n_nodes, 'node_id': top_n_node_ids, 'IAB_Candidates': top_n_IAB_Candidates,
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new, "H_Mapping_scores": top_n_H_scores }

In [105]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [106]:
df_MAF_new.to_excel('results/OpenAI_results_0.755_new.xlsx')

In [107]:
df_MAF_new.shape

(77838, 6)

In [ ]:
#Top 100 in predictions but not in embedding

In [108]:
df_MAF_new=df_MAF_new.sort_values(by='cosine_predicted', ascending=False)

In [109]:
df_MAF_new.iloc[0]

item                                       minivan
node_id                                       4867
IAB_Candidates      ['Auto Parts', 'Auto Rentals']
predictions                                Minivan
cosine_predicted              [0.9860634986725173]
H_Mapping_scores        [0.724198355, 0.730903592]
Name: 37363, dtype: object

In [110]:
top_100_node=[]
top_100_embedding_pred=[]
top_100_H_pred=[]
top_100_cos_pred=[]
top_100_H_score=[]

In [111]:
items=df_MAF_new['item'].to_list()
IAB_Candidates=df_MAF_new['IAB_Candidates'].to_list()
predictions=df_MAF_new['predictions'].to_list()
cosine_predicted=df_MAF_new['cosine_predicted'].to_list()
H_Mapping_scores=df_MAF_new['H_Mapping_scores'].to_list()

In [112]:
for item,IAB,pred,cos,H_map in tqdm(zip(items,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores)):
    pred_list=ast.literal_eval(IAB)
    if pred not in pred_list:
        top_100_node.append(item)
        top_100_embedding_pred.append(pred)
        top_100_H_pred.append(IAB)
        top_100_cos_pred.append(cos)
        top_100_H_score.append(H_map)
    if len(top_100_node)==100:
        break

253it [00:00, 19364.57it/s]


In [113]:
top_100_dic={'items':top_100_node, 'Embedding Predictions': top_100_embedding_pred,
 "H_Mapping_Pred": top_100_H_pred, 'Predicted Cosine': top_100_cos_pred, 
 'H Mapping Cosine': top_100_H_score}

In [114]:
df_top_100=pd.DataFrame.from_dict(top_100_dic)

In [115]:
df_top_100.to_excel('results/OpenAI_Top_100_Not_In_H.xlsx', index=False)

In [ ]:
#Top 100 in H but not in Embedding

In [116]:
print(top_n_labels[5])
print(top_n_cosines[5])

['Dogs', 'SUV', 'Cats', 'Large Animals', 'Sedan', 'Pets', 'Polar Travel', 'Camping', 'Beauty', 'Desktops', 'Wellness', 'Science', 'Minivan', 'Reptiles', 'Pickup Trucks', 'Laptops', 'Sports', 'Travel', 'Outdoor Activities', 'Hair Care', 'Hobbies & Interests', 'Birds', 'Cruises', 'Parenting', 'Dating', 'Convertible', 'Games and Puzzles', 'Movies', 'Shaving', 'Smartphones', 'Motorcycles', 'Shopping', 'Graduation', 'Cooking', 'Parks & Nature', 'Holiday Shopping', 'Space and Astronomy', 'Luxury Cars', 'Healthy Living', 'Single Life', 'Education', 'Commercial Trucks', 'Rail Travel', 'Houses', 'Station Wagon', 'Wedding', 'Travel Locations', "Women's Handbags and Wallets", 'Coupe', 'Travel Accessories', 'Beach Travel', 'Apartments', 'Grocery Shopping', 'Asia Travel', 'South America Travel', 'Board Games and Puzzles', 'Barbecues and Grilling', 'Bath and Shower', 'Europe Travel', 'Landscaping', 'Fitness and Exercise', 'Home Security', 'Smart Home', 'Air Travel', 'Home & Garden', 'Gardening', 'Zo

In [117]:
nodes=df_MAF['item'].to_list()
# node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
H_Mappings=df_MAF['H_Scores'].to_list()

In [118]:
mid_node=[]
mid_IAB=[]
mid_label=[]
mid_cos=[]
mid_H_map=[]

for node, IAB, label, cos, H_map in tqdm(zip(nodes,IAB_Candidates,top_n_labels,top_n_cosines,H_Mappings)):
    IAB_list=ast.literal_eval(IAB)
    H_Map_list=ast.literal_eval(H_map)
    for IAB_item, H_Map_item in zip(IAB_list,H_Map_list):
        mid_node.append(node)
        mid_IAB.append(IAB_item)
        mid_label.append(label)
        mid_cos.append(cos)
        mid_H_map.append(H_Map_item)

942it [00:00, 34295.68it/s]


In [119]:
dic={'item':mid_node, 'IAB_Candidates': mid_IAB,
'predictions': mid_label, 'cosine_predicted': mid_cos, "H_Mapping_scores": mid_H_map}

In [120]:
df_lol_100=pd.DataFrame.from_dict(dic)

In [121]:
df_lol_100=df_lol_100.sort_values(by='H_Mapping_scores', ascending=False)

In [122]:
df_lol_100.head()

item          IAB_Candidates  \
2613      wagon                  Travel   
2614       drop             Real Estate   
2615       drop                  Travel   
2629  mousetrap           Home & Garden   
2626  ring_mail  Events and Attractions   

                                            predictions  \
2613  [Station Wagon, Minivan, SUV, Convertible, Tra...   
2614  [Dating, Travel, Graduation, Movies, Shopping,...   
2615  [Dating, Travel, Graduation, Movies, Shopping,...   
2629  [Cats, Movies, Coupe, Birds, Games and Puzzles...   
2626  [Rail Travel, Travel, Birds, Nail Care, Hair C...   

                                       cosine_predicted  H_Mapping_scores  
2613  [[0.8676690064930836], [0.8482987892336509], [...               1.0  
2614  [[0.8176049105530151], [0.8087438873386152], [...               1.0  
2615  [[0.8176049105530151], [0.8087438873386152], [...               1.0  
2629  [[0.7973278023641381], [0.7922019973205039], [...               1.0  
2626  [[0.7915800550643299], [0.7658795812872278], [...               1.0

In [123]:
top_100_node=[]
top_100_embedding_pred=[]
top_100_H_pred=[]
top_100_cos_pred=[]
top_100_H_score=[]

In [124]:
items=df_lol_100['item'].to_list()
IAB_Candidates=df_lol_100['IAB_Candidates'].to_list()
predictions=df_lol_100['predictions'].to_list()
cosine_predicted=df_lol_100['cosine_predicted'].to_list()
H_Mapping_scores=df_lol_100['H_Mapping_scores'].to_list()

In [125]:
for item,IAB,pred,cos,H_map in tqdm(zip(items,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores)):
    #pred_list=ast.literal_eval(pred)
    if IAB not in pred:
        top_100_node.append(item)
        top_100_embedding_pred.append(pred)
        top_100_H_pred.append(IAB)
        top_100_cos_pred.append(cos)
        top_100_H_score.append(H_map)
    if len(top_100_node)==100:
        break

1311it [00:00, 678855.87it/s]


In [126]:
top_100_dic={'items':top_100_node, 'Embedding Predictions': top_100_embedding_pred,
 "H_Mapping_Pred": top_100_H_pred, 'Predicted Cosine': top_100_cos_pred, 
 'H Mapping Cosine': top_100_H_score}

In [127]:
df_top_100=pd.DataFrame.from_dict(top_100_dic)

In [128]:
df_top_100.to_excel('results/Top_100_Not_In_OpenAI.xlsx', index=False)